# Cuaderno de ingesta de datos

En este bloque traeremos desde datos abiertos.

In [0]:
# Paso 1: Descargar los datos y leerlos en pandas, luego convertir a Spark

import requests
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession

# Iniciar sesión Spark
spark = SparkSession.builder.getOrCreate()

# URLs de los datasets
url_secop = "https://www.datos.gov.co/resource/rpmr-utcd.csv?$limit=100000"
url_men = "https://www.datos.gov.co/resource/nudc-7mev.csv?$limit=100000"

# Función para descargar y leer CSV desde la web
def descargar_csv(url):
    try:
        response = requests.get(url, timeout=10)  # Evita bloqueo por espera larga
        response.raise_for_status()  # Lanza error si el código de estado no es 200
        return pd.read_csv(StringIO(response.text))
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar los datos desde {url}:\n{e}")
        return pd.DataFrame()  # Retorna un DataFrame vacío si falla

# Descargar y leer en pandas
df_secop_pd = descargar_csv(url_secop)
df_men_pd = descargar_csv(url_men)

# Verificar si los DataFrames no están vacíos antes de convertir
if not df_secop_pd.empty and not df_men_pd.empty:
    # Convertir a DataFrame Spark
    df_secop = spark.createDataFrame(df_secop_pd)
    df_men = spark.createDataFrame(df_men_pd)

    # Mostrar en Databricks
    display(df_secop)
    display(df_men)
else:
    print("Alguno de los DataFrames está vacío. Verifica la conexión o URLs.")

In [0]:
# Celda 1: Leer datos desde los archivos CSV que subiste a Volumes

# Rutas locales dentro del entorno Databricks (Volumes)
url_secop = "/Volumes/main/diplomado_datos/manual/df_secop.csv"
url_men = "/Volumes/main/diplomado_datos/manual/df_men.csv"

# Leer los archivos usando Spark
# "header" indica que los nombres de las columnas están en la primera fila
# "inferSchema" permite que Spark adivine automáticamente los tipos de datos
df_secop = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(url_secop)
df_men = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(url_men)

# Mostrar los primeros registros en Databricks usando .show()
print("Datos del SECOP cargados:")
df_secop.show()

print("Datos del MEN cargados:")
df_men.show()

In [0]:
display(df_men.limit(10))
display(df_secop.limit(10))

In [0]:
df_secop.count()
df_men.count()

In [0]:
# Celda 2: Guardar los DataFrames como tablas Delta

# La función .saveAsTable() guarda los datos y registra la tabla en el Unity Catalog.
# El modo "overwrite" reemplaza la tabla si ya existe, ideal para actualizaciones.

df_secop.write.format("delta").mode("overwrite").saveAsTable("main.diplomado_datos.secop")
df_men.write.format("delta").mode("overwrite").saveAsTable("main.diplomado_datos.men_estadisticas")

print("¡Tablas guardadas exitosamente en el catálogo 'main', esquema 'diplomado_datos'!")

In [0]:
import requests
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession

# Crear sesión de Spark
spark = SparkSession.builder.getOrCreate()

# URL del dataset
url_secop = "https://www.datos.gov.co/resource/rpmr-utcd.csv?$limit=100000&$offset=100000"

# Descargar contenido
response_secop = requests.get(url_secop)

# Leer CSV en pandas
df_secop_pd = pd.read_csv(StringIO(response_secop.text), delimiter=",", low_memory=False)

# Limpiar nombres de columnas (opcional pero recomendado)
df_secop_pd.columns = [col.strip().lower().replace(" ", "_") for col in df_secop_pd.columns]

# Verifica que los datos se cargaron correctamente en pandas
print(df_secop_pd.head())

# Convertir a Spark DataFrame
df_secop = spark.createDataFrame(df_secop_pd)

# Mostrar en Databricks
display(df_secop)

## DataSets

In [0]:
from pyspark.sql.functions import col

# Get the target schema
target_schema = spark.table("main.diplomado_datos.secop").schema

# Select and cast columns that exist in both df_secop and target_schema
df_secop_aligned = df_secop.select(
    [col(field.name).cast(field.dataType) for field in target_schema.fields if field.name in df_secop.columns]
)

# Write the aligned DataFrame to the Delta table
df_secop_aligned.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable("main.diplomado_datos.secop")

In [0]:
total_registros = 19446266
offset_inicial = 200000
limite = 100000
paginas_faltantes = ((total_registros - offset_inicial) // limite) + 1

print(f"Quedan {paginas_faltantes} bloques por descargar...")